<a href="https://colab.research.google.com/github/annieco29/wid_climate_change_datathon/blob/master/notebooks/transformers_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00


In [2]:
from transformers import pipeline
import pandas as pd

# test the Twitter-roberta-base-sentiment model using transformers
sentiment_task = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest")
sentiment_task("Covid cases are increasing fast!")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'negative', 'score': 0.7235764265060425}]

In [3]:
## Note: This data was updated to Colab manually from my local computer, and is available for download here: https://github.com/annieco29/wid_climate_change_datathon/tree/master/data
# Read in data
df = pd.read_csv("bbc_and_nasa_articles_locations_coordinates.csv")

In [4]:
df.head()

,Unnamed: 0,city,state_name,county_name,lat,lng,zips,title,link,description,published,location_match
0,0,New York,New York,Queens,40.6943,-73.9249,11229 11226 11225 11224 11223 11221 11220 1138...,How ancient collision shaped New York skyline,http://www.bbc.co.uk/news/science-environment-...,How the collision of ancient landmasses framed...,"Fri, 07 Jun 2013 01:26:25 GMT",New York
1,1,New York,New York,Queens,40.6943,-73.9249,11229 11226 11225 11224 11223 11221 11220 1138...,New York 'at risk' as seas rise,http://www.bbc.co.uk/go/rss/int/news/-/news/sc...,New York is set to be a major loser as a resul...,"Fri, 08 Apr 2011 12:00:28 GMT",New York
2,2,New York,New York,Queens,40.6943,-73.9249,11229 11226 11225 11224 11223 11221 11220 1138...,VIDEO: Get up close to shuttle Enterprise,http://www.bbc.co.uk/news/science-environment-...,Nasa's shuttle Enterprise has moved to a new h...,"Thu, 19 Jul 2012 12:53:27 GMT",New York
3,3,New York,New York,Queens,40.6943,-73.9249,11229 11226 11225 11224 11223 11221 11220 1138...,VIDEO: 'Bio-hackers' create New York lab,http://www.bbc.co.uk/go/rss/int/news/-/news/wo...,A group of scientists have created the first c...,"Wed, 15 Feb 2012 03:26:00 GMT",New York
4,4,New York,New York,Queens,40.6943,-73.9249,11229 11226 11225 11224 11223 11221 11220 1138...,Lab rodents drowned in NY floods,http://www.bbc.co.uk/news/science-environment-...,Thousands of lab rodents have drowned after st...,"Thu, 01 Nov 2012 15:43:51 GMT",New York


In [5]:
# sample dataframe to test code
df_sample = df.head()

In [6]:
# Define the functions

# Function to get sentiment
def get_sentiment_label(text):
    # Process text with roBERTa
    doc = sentiment_task(text)
    doc_dict = doc[0]
    # Return the polarity
    return doc_dict['label']

def get_sentiment_score(text):
    # Process text with roBERTa
    doc = sentiment_task(text)
    doc_dict = doc[0]
    # Return the polarity
    return doc_dict['score']

def get_sentiment_label_and_score(text):
    label, score = get_sentiment_label(text), get_sentiment_score(text)
    return label, score

# Apply the functions to the 'title' and 'description' columns
result_columns = df[['title', 'description']].applymap(get_sentiment_label_and_score)

# Rename the resulting columns
result_columns.columns = ['roberta_title', 'roberta_description']

# Concatenate the result_columns DataFrame with the original df
df_sentiment = pd.concat([df, result_columns], axis=1)

# The df DataFrame now contains 'roberta_title' and 'roberta_description' columns


In [7]:
# Split the label from the score in the output model result columns
df_sentiment[['roberta_title_label','roberta_title_score']] = df_sentiment['roberta_title'] .apply(lambda x: pd.Series({'label': x[0], 'score': x[1]}))
df_sentiment[['roberta_desc_label','roberta_desc_score']] = df_sentiment['roberta_description'] .apply(lambda x: pd.Series({'label': x[0], 'score': x[1]}))

In [8]:
df_sentiment[['roberta_title_label','roberta_title_score','roberta_desc_label','roberta_desc_score']]

,roberta_title_label,roberta_title_score,roberta_desc_label,roberta_desc_score
0,neutral,0.882552,neutral,0.917406
1,neutral,0.708405,negative,0.838716
2,neutral,0.886371,neutral,0.919762
3,neutral,0.860775,positive,0.670368
4,negative,0.530886,negative,0.803492
...,...,...,...,...
441,neutral,0.837172,neutral,0.641307
442,neutral,0.832395,neutral,0.686284
443,neutral,0.888790,negative,0.730751
444,negative,0.492985,neutral,0.918512


In [9]:
df_sentiment[['roberta_title_label']].value_counts()

roberta_title_label
neutral                360
negative                68
positive                18
dtype: int64

In [10]:
df_sentiment[['roberta_title_score']].describe()

,roberta_title_score
count,446.000000
mean,0.761968
std,0.126385
min,0.482474
25%,0.662557
50%,0.789273
75%,0.871505
max,0.949781


In [11]:
df_sentiment[['roberta_desc_label']].value_counts()

roberta_desc_label
neutral               297
negative              111
positive               38
dtype: int64

In [12]:
df_sentiment[['roberta_desc_score']].describe()

,roberta_desc_score
count,446.000000
mean,0.754912
std,0.135914
min,0.500273
25%,0.637308
50%,0.779406
75%,0.878340
max,0.975464


In [13]:
df_sentiment.to_csv("bbc_and_nasa_articles_loc_coor_sent.csv")